In [4]:
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [61]:
# Check data
mat = sp.io.loadmat("dataset1.mat")

mat.keys()
# '__header__', '__version__', '__globals__',
# 'r', 'x_true', 't', 'v', 'l', 'r_var', 'v_var'

r = mat["r"]
x_true = mat["x_true"]
t = mat["t"]
v = mat["v"]
l = mat["l"]
r_var = mat["r_var"]
v_var = mat["v_var"]
# print(r.shape, x_true.shape, t.shape, v.shape, l.shape, r_var.shape, v_var.shape)

# (1,) -> (12709,)
l = np.broadcast_to(l, r.shape)
r_var = np.broadcast_to(r_var, r.shape)
v_var = np.broadcast_to(v_var, r.shape)

col_name = ["r", "x_true", "t", "v", "l", "r_var", "v_var"]
df = pd.DataFrame(np.hstack([r, x_true, t, v, l, r_var, v_var]), columns=col_name)
df.head()

,r,x_true,t,v,l,r_var,v_var
0,3.454506,0.984457,0.0,0.0,4.428479,0.000367,0.002261
1,3.456506,0.984434,0.1,0.0,4.428479,0.000367,0.002261
2,3.452506,0.984518,0.2,0.0,4.428479,0.000367,0.002261
3,3.454506,0.984452,0.3,0.0,4.428479,0.000367,0.002261
4,3.451506,0.984406,0.4,0.0,4.428479,0.000367,0.002261


In [98]:
# load data
mat = sp.io.loadmat("dataset1.mat")
r = mat["r"].reshape(-1)
x_true = mat["x_true"].reshape(-1)
t = mat["t"].reshape(-1)
v = mat["v"].reshape(-1)
l = mat["l"].reshape(-1)
r_var = mat["r_var"].reshape(-1)
v_var = mat["v_var"].reshape(-1)


K = x_true.shape[0]  # 12709
T = 0.1

delta = 1000
k = K // delta  # number of sub-samples

# make z
## make u
u = np.zeros((k, 1))
for i in range(1, k):
    u[i] = T * v[(i - 1) * delta : i * delta].sum()

## make y
y = np.zeros((k, 1))
for i in range(1, k):
    y[i] = l[0] - r[i * delta]

## make z
z = np.vstack([u, y])

# make H
## make A_inv
A_inv = np.eye(k)
for row in range(A_inv.shape[0]):
    A_inv[row, row - 1] = -1

## make C
C = np.eye(k)

## make H
H = np.vstack([A_inv, C])

# make W
## make Q, R
Q = np.eye(k) * v_var[0]
R = np.eye(k) * r_var[0]

## make W
W = np.block([[Q, np.zeros((k, k))], [np.zeros((k, k)), R]])


# for i in range(1, num_sample, delta):
#     print(i)